In [35]:
#import libraries
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import scipy

#LDPC codes

##1.

Write a function that receives a parity check matrix H and builds a systematic encoding matrix G for it. The function should return two matrices: ˆH and G, such that ˆH is equal to H up to a column permutation and ˆHGt = 0 for all t.

In [70]:
#initialize H
H = np.array([[1, 1, 1, 1, 0, 0],[0, 0, 1, 1, 0, 1],[1, 0, 0, 1, 1, 0]], dtype=int)

print(H)


[[1 1 1 1 0 0]
 [0 0 1 1 0 1]
 [1 0 0 1 1 0]]


In [87]:
def rref(H):

    ''' return reduce row echelon form'''
    m, n = H.shape
    # k = n-m

    i = 0
    j = 0
    r = 0

    H_hat = H.copy()

    while True:

        if i >= m or j >= n:
            break
        
        # print(H_hat)
        
        #check if entry = 0
        if H_hat[i,j]== 0:

            #find next row with non zero entry to swap
            r = i

            while r < m and H_hat[r,j] == 0 :
                r += 1
            
            #if no nonzero found skip to next column
            if r == m:
                j+=1
                break
        
        #swap row
        zero_row = H_hat[i,:].copy()
        nonzero = H_hat[r,:].copy()
        H_hat[i,:] = nonzero
        H_hat[r,:] = zero_row
        
        # print(H_hat)

        #perform row elimination
        for row in range(m):
            if row == i:
                continue
            #if entry does not = 0, add nonzero row, remember this is in F2 (so mod 2 addition)
            if H_hat[row,j] != 0:
                H_hat[row,:] = (H_hat[row,:] + H_hat[i,:]) % 2
        
        # print(H_hat)

        i+=1
        j+=1
    
    return H_hat


In [139]:
def gen_G(H):
    m, n = H.shape
    k = n-m

    H_hat = rref(H)
    # print("H_hat:\n",H_hat)
    P = H_hat[:, m:n]
    # print("P:\n",P)
    G = np.vstack((P, np.identity(k)))
    # print("G\n",G) 

    return H_hat, G
    

In [141]:
H_hat, G = gen_G(H)

print("H_hat:\n",H_hat, "\nG:\n", G)

H_hat:
 [[1 0 0 1 1 0]
 [0 1 0 1 1 1]
 [0 0 1 1 0 1]] 
G:
 [[1. 1. 0.]
 [1. 1. 1.]
 [1. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


##3.

Write an LDPC-decoder based on Loopy Belief Propagation for Binary Symmetric Channel.

In [201]:
#import y1
file_y = open("y1.txt")
y_content = file_y.read()
file_y.close()
y = y_content.split("\n")
# print(y1)
y = y[:-1]
y1 = np.array([int(i) for i in y])

y1.shape

(1000,)

In [205]:
#import H1
file_h = open("H1.txt")
H_content = file_h.read()
H1 = H_content.split("\n")
H1 = H1[:-1]
H = [np.array(i.split(), dtype=int) for i in H1]
h1 = np.vstack(H)

h1.shape


(750, 1000)

In [ ]:
def decode(H, y, p=.1, max_iter = 20):

    iter = 0
    m, n = H.shape
    k = n-m

    #initialize
    x = y.copy()

    for j in m:
        for i in n:

    